In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import *
from lightgbm import LGBMRegressor
import pickle
import time
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [2]:
start = time.time()
with open('dicts/users_dict_all.pkl', 'rb') as f:
    users_dic = pickle.load(f)
with open('dicts/group_dict_all.pkl', 'rb') as f:
    groups_dic = pickle.load(f)
groups_dic['1']

{'sum': 241, 'number': 8}

In [3]:
for key in groups_dic:
    groups_dic[key]['avg'] = groups_dic[key]['sum'] / groups_dic[key]['number']

for key in tqdm(users_dic):
    users_dic[key]['friends_ages'] = [users_dic[str(fuid)]['age'] for fuid in users_dic[key]['friends'] if users_dic[str(fuid)]['age'] > 0]
    users_dic[key]['friends_avg_ages'] = np.mean(users_dic[key]['friends_ages'])
    users_dic[key]['groups_ages'] = [groups_dic[str(gid)]['avg'] for gid in users_dic[key]['groups'] if groups_dic[str(gid)]['number'] > 0]
    users_dic[key]['groups_avg_ages'] = np.mean(users_dic[key]['groups_ages'])

for key in tqdm(users_dic):
    users_dic[key]['friends_of_friends_avg_age'] = np.mean([users_dic[str(fuid)]['friends_avg_ages'] for fuid in users_dic[key]['friends'] if users_dic[str(fuid)]['friends_avg_ages'] > 0])
    users_dic[key]['friends_of_friends_median_age'] = np.median([users_dic[str(fuid)]['friends_avg_ages'] for fuid in users_dic[key]['friends'] if users_dic[str(fuid)]['friends_avg_ages'] > 0])
print(users_dic['1'])
print(time.time() - start)

E:\programs\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
E:\programs\Anaconda\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



{'age': 34, 'registered_year': 4, 'school_education': 35.0, 'graduation_1': nan, 'graduation_2': nan, 'graduation_3': nan, 'graduation_4': nan, 'graduation_5': nan, 'graduation_6': nan, 'graduation_7': nan, 'friends': [25478, 8920, 35884, 9155, 8221, 30276, 24231, 113282, 15975, 8623, 86326, 58325, 1286, 3897, 8112, 56700, 40083, 79883, 23937, 111914, 116849, 74104, 108059, 84485, 118301, 45396, 107129, 113595, 54910, 111764, 114837, 60301, 111436, 274, 74104, 67135, 1286, 119099, 35884, 89538, 103756, 21708, 353, 73169, 89826, 50497, 101936, 53319, 24788, 86277, 42464, 77473, 111914, 38642, 35428, 85654, 30379, 16812, 112614, 18528, 114583, 7784, 39397, 12052, 35385, 8920, 58325, 19068, 30526, 100966, 101796, 23315, 114319, 52867, 48794, 13682, 7910, 114266, 107077, 73083, 119464, 71012, 116849, 274, 23655, 94853, 118230, 58896, 116377, 70408, 76395, 13939, 43818, 238, 73169, 39758, 6535, 13781, 118230, 57314, 48794, 29886, 107129, 77449, 108059, 98579, 23315, 52506, 28241, 44860, 97

In [4]:
train_df = pd.read_csv('data_full/train.csv')
uids = train_df.uid.values

In [5]:
ages, reg_years, sch_ed, gr1 = [], [], [], []
gr2, gr3, gr4, gr5 = [], [], [], []
gr6, gr7, avg_fr, avg_gr = [], [], [], []
num_fr, num_gr, avg_reg_years = [], [], []
median_fr, median_gr = [], []
av_friends_of_friends = [] # средний возраст друзей друзей
median_friends_of_friends = [] # медианный возразт друзей друзей
av_groups_of_friends = [] # средний возраст групп друзей
median_groups_of_friends = []
for uid in tqdm(uids):
    uid = str(uid)
    av_friends_of_friends.append(users_dic[uid]['friends_of_friends_avg_age'])
    median_friends_of_friends.append(users_dic[uid]['friends_of_friends_median_age'])
    regs = [users_dic[str(fuid)]['registered_year'] for fuid in users_dic[uid]['friends'] if users_dic[str(fuid)]['registered_year'] > 0]
    avg_reg_years.append(np.average(regs) if len(regs) > 0 else np.nan)
    ages.append(users_dic[uid]['age'])
    reg_years.append(users_dic[uid]['registered_year'])
    sch_ed.append(users_dic[uid]['school_education'])
    gr1.append(users_dic[uid]['graduation_1'])
    gr2.append(users_dic[uid]['graduation_2'])
    gr3.append(users_dic[uid]['graduation_3'])
    gr4.append(users_dic[uid]['graduation_4'])
    gr5.append(users_dic[uid]['graduation_5'])
    gr6.append(users_dic[uid]['graduation_6'])
    gr7.append(users_dic[uid]['graduation_7'])
    avg_fr.append(np.average(users_dic[uid]['friends_ages']) if len(users_dic[uid]['friends_ages']) > 0 else np.nan)
    median_fr.append(np.median(users_dic[uid]['friends_ages']) if len(users_dic[uid]['friends_ages']) > 0 else np.nan)
    avg_gr.append(np.average(users_dic[uid]['groups_ages']) if len(users_dic[uid]['groups_ages']) > 0 else np.nan)
    median_gr.append(np.median(users_dic[uid]['groups_ages']) if len(users_dic[uid]['groups_ages']) > 0 else np.nan)
    num_fr.append(len(users_dic[uid]['friends']))
    num_gr.append(len(users_dic[uid]['groups']))
    av_groups_of_friends.append(np.average([users_dic[str(fuid)]['groups_avg_ages'] for fuid in users_dic[uid]['friends'] if users_dic[str(fuid)]['groups_avg_ages'] > 0]))
    median_groups_of_friends.append(np.median([users_dic[str(fuid)]['groups_avg_ages'] for fuid in users_dic[uid]['friends'] if users_dic[str(fuid)]['groups_avg_ages'] > 0]))
train = pd.DataFrame({
    'age': ages,
    'registered_year': reg_years,
    'school_ed': sch_ed,
    'gr_1': gr1,
    'gr_2': gr2,
    'gr_3': gr3,
    'gr_4': gr4,
    'gr_5': gr5,
    'gr_6': gr6,
    'gr_7': gr7,
    'num_friends': num_fr,
    'av_reg_year': avg_reg_years,
    'median_friends_age': median_fr,
    'avg_friends_age': avg_fr,
    'num_groups': num_gr,
    'avg_groups_age': avg_gr,
    'median_groups_age': median_gr,
    'av_friends_of_friends': av_friends_of_friends,
    'median_friends_of_friends': median_friends_of_friends,
    'av_groups_of_friends': av_groups_of_friends,
})
train

E:\programs\Anaconda\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)


,age,registered_year,school_ed,gr_1,gr_2,gr_3,gr_4,gr_5,gr_6,gr_7,num_friends,av_reg_year,median_friends_age,avg_friends_age,num_groups,avg_groups_age,median_groups_age,av_friends_of_friends,median_friends_of_friends,av_groups_of_friends
0,32,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,6.500000,35.5,36.428571,4,36.746115,37.847786,35.139755,35.258621,35.064582
1,30,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126,6.310345,33.0,34.068966,100,34.821975,34.390129,33.690534,33.860390,34.697590
2,51,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,4.333333,32.0,35.000000,0,NaN,NaN,37.949089,37.622742,34.340875
3,17,14,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,0,NaN,NaN,16.833333,16.833333,NaN
4,33,5,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,4.166667,34.0,35.000000,0,NaN,NaN,34.549592,34.146341,33.978508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29464,34,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,8.692308,31.0,29.846154,0,NaN,NaN,33.957473,34.096908,31.289558
29465,39,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,7.400000,37.0,42.400000,0,NaN,NaN,33.587969,33.038462,35.261726
29466,44,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,7.666667,38.0,38.000000,0,NaN,NaN,35.976041,35.568182,35.788775
29467,38,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,118,7.478261,38.0,39.304348,10,33.545356,33.461951,37.242517,37.210714,35.869598


In [6]:
X, y = train.drop('age', axis=1), train['age']

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    train_dataset = Pool(X.iloc[train_index], y.iloc[train_index])
    test_dataset = Pool(X.iloc[test_index], y.iloc[test_index])
    
    model = CatBoostRegressor(
        iterations = 350,
        depth = 8,
        random_seed = 1000
    )
    model.fit(train_dataset, eval_set=test_dataset, silent=True)
    preds = model.predict(test_dataset)
    print(mean_squared_error(y.iloc[test_index], preds, squared=False))
    with open(f'dicts/model_{i}.cbm', 'wb') as f:
        pickle.dump(model, f)
preds = model.predict(X)

6.307399501144355
6.3945388404043095
6.537816856105048
6.149106686696097
6.095908859791258


In [47]:
X, y = train.drop('age', axis=1), train['age']
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]
    model = LGBMRegressor(
        objective='regression',
        learning_rate=0.1,
        max_bin=1000
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(mean_squared_error(y_test, preds, squared=False))
    with open(f'dicts/model_{i}.lgbm', "wb") as f:
        pickle.dump(model, f)

6.295148903715391
6.463224239240568
6.564780593945442
6.190186487922053
6.155565317479468
